Loading and exploring dataset

In [1]:
%load_ext autoreload
%autoreload 2

Importing pandas and numpy

In [2]:
import pandas as pd
import numpy as np

Loading training set into dataframe called df

In [3]:
df = pd.read_csv('../data/raw/train.csv',low_memory=False)

Splitting data into features and target

In [5]:
features = df.drop(['player_id','drafted'],axis=1)
target = df['drafted']

One-hot-encoding

In [6]:
features = pd.get_dummies(features)

In [7]:
print(features.shape,target.shape)

(56091, 542) (56091,)


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

Training the model

In [21]:
from xgboost import XGBClassifier

In [22]:
# Model training (XGBoost)
xgb = XGBClassifier()
xgb.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [23]:
from sklearn.metrics import roc_auc_score

In [24]:
# Model evaluation (Random Forest)
y_pred_prob = xgb.predict_proba(X_val_scaled)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_prob)
print(f'XGBoost AUROC: {roc_auc:.4f}')

XGBoost AUROC: 0.9987


In [25]:
test_data = pd.read_csv('../data/raw/test.csv',low_memory=False)

In [28]:
test_data_processed = pd.get_dummies(test_data)
test_data_processed = test_data_processed.reindex(columns=features.columns, fill_value=0)  # Align columns
test_data_scaled = scaler.transform(test_data_processed)
predictions = xgb.predict_proba(test_data_scaled)[:,1]

In [29]:
test_data['drafted'] = predictions
print(test_data[['player_id', 'drafted']])

                                 player_id       drafted
0     cf302b4d-84f7-4124-a25d-a75eed31978b  2.807759e-06
1     f91837cd-4f49-4b70-963d-aeb82c6ce3da  3.906363e-06
2     53ec2a29-1e7d-4c6d-86d7-d60d02af8916  2.285963e-05
3     32402798-471c-4a54-8cb4-29cd95199014  9.065740e-06
4     73b960f9-27b8-4431-9d23-a760e9bbc360  7.091778e-06
...                                    ...           ...
4965  a25ee55f-02a3-4f8e-8194-a5f427e14e7c  1.756786e-06
4966  d0d9f45e-7b01-44b3-8d40-514ec338611d  1.064129e-06
4967  f8df22c4-1602-4fab-896d-8820951aae2f  9.939547e-07
4968  b791c69a-f769-4163-afda-051a6fd20a9d  9.939547e-07
4969  18b51f5d-4746-4121-88fd-c8d0a1399130  3.039967e-06

[4970 rows x 2 columns]


In [30]:
test_data[['player_id', 'drafted']].to_csv('predictions_xgb.csv',index=False)